In [33]:

#Importar librerias? Pues claro que me importan!
import numpy as np
from numpy import random
from random import choices 
     

In [34]:
#Segun lo buscado, el periodo de incubacion puede ir de los 1 a los 12 dias
#con una probabilidad del 50% que termine la incubacion a los 5 dias
def tiempo_incubacion():
    dias = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12]
    peso_de_decision = [0.045, 0.045, 0.045, 0.045, 0.5, 0.045, 0.045, 0.045, 0.045, .045, 0.045, 0.045]
    n = choices(dias, peso_de_decision)
    return n

In [51]:
def asintomatico_o_no():
    estado = ["asintomatico", "sintomatico"]
    peso_de_decision = [0.8, 0.2]
    n = choices(estado, peso_de_decision)
    return n

In [52]:
def probabilidad_de_muerte(edad):
    if(edad > 65):
        valoracion = [1,0]
        peso = [0.7, 0.3] #queda pendiente revisar esto 
        n = choices(valoracion, peso)
    elif(edad < 65):
        valoracion = [1,0]
        peso = [0.045, 0.955] #probabilidad que se correspende en caso que no tenga antecentes de ningun tipo
        n = choices(valoracion, peso)
    return n
    

In [53]:
#Objeto que simula a una persona.
class Persona:
        def __init__(self, idpersona,edad,lugar, dni):           
            self.idpersona=idpersona
            self.edad=edad
            self.estadosposibles=["sano","incubando","incubandoContagioso","sintomatico","asintomatico","muerto","inmune"]
            self.estado=self.estadosposibles[0]
            self.lugar=lugar
            self.diasestado=0
            self.cambioEstado=np.inf  #Atributo que indica que dia cambiara la persona de estado.
            self.contadorInfecciones=0
            self.dni = dni
            
            
           
        def __str__(self):
            #Devuelve un string con la informacion util del sujeto
            return str(self.idpersona) + "," + str(self.estado) + "," + str(self.lugar) + "," +str(self.dni)         

        def puede_propagar(self):
            return ((self.estado == "sintomatico") or (self.estado == "asintomatico") or (self.estado == "incubandoContagioso"))
        def susceptible(self):
            return (self.estado == "sano")
        
        def infectar(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion): #media y desv covid: 7.5 y 3.5
            if self.puede_propagar():
                for i in range(len(ciudad[self.lugar])): 
                    if (ciudad[self.lugar][i].susceptible):
                        nocontagia=random.uniform(0, 1)   #Numero al azar, si es menor que la pos. de contagio, contagiara
                        if (nocontagia <posibilidadContagio/24):
                            ciudad[self.lugar][i].estado=ciudad[self.lugar][i].estadosposibles[1]
                            ciudad[self.lugar][i].diasestado=0
                            ciudad[self.lugar][i].cambioEstado=random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))
                            self.contadorInfecciones +=1
        
        #Se le pasa como parametro la propia persona, dia de simulacion
        def transicionEstados(self, dia, mediaincubacion,desvincubacion):
            #A partir de aqui se comprueba que estado posee la persona
            #Y se mira dos casos:
            #Si el dia del cambio de estado , coincide con el dia actual
            if(self.estado == "sintomatico"):
                me_voy_a_morir = probabilidad_de_muerte(self.edad)

                if(me_ve_voy_a_morir == 1):
                    self.cambioEstado=np.inf
                    self.estado = "muerto"
            
            if(self.cambioEstado == dia):
                if(self.estado == "incubando"):
                    self.estado = "incubandoContagioso" #se le asigna el siguiente estado  
                    self.cambioEstado = random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2 + dia   #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "incubandoContagioso"):
                    self.estado = asintomatico_o_no() #se le asigna el siguiente estado 
                    self.cambioEstado = 15 + dia #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "sintomatico"):
                    self.cambioEstado = np.inf 
                    self.estado = "inmune"
                    '''
                      Y ahora una vez que pasa a ser asintomatico o no, se debe ver la probabilidad de que la persona se vaya 
                       a morir o no, y dependiendo de eso, se ha asignan unos dias...
                    '''

                elif(self.estado == "asintomatico"):
                    self.estado = "inmune"
                    self.cambioEstado = np.inf 


            
                

In [38]:
#Funcion que se encarga de generar la edad de manera aleatoria
def generador_edad():
        n = random.randint(0,110) #el rango de edad, sera desde los 0 años hasta los 110
        return n


In [24]:
#Funcion que se encarga de generar numeros aleatorios y los adiciona a una lista
def generador_numeros():
    randomlist = []
    for i in range(0,8): #genera 8 numeros aleatorios
        n = random.randint(0,9) #los numeros alteatorios son entre el 0 y el 9
        randomlist.append(n) #una vez generado el numero aleatorio, se adiciona a la lista
    return randomlist


In [25]:
#Funcion que se encarga de crear una secuencia valida de DNI
#Para ello, se ha utilizado el algoritmo que se usa en la vida real -en cuanto a la relacion numero y letra- 
def creaDni(input):
    letras = "TRWAGMYFPDXBNJZSQVHLCKEO" #conjunto de letras del dni

    #Se consigue que toda la secuencia de numeros sea 1 solo
    strings = [str(integer) for integer in input ]
    a_string = "".join(strings)
    my_seq = int(a_string)
    #print(f"El valor de la secuencia de numeros es {my_seq}")

    #Se calcula la letra que ha de ser adicionada
    valor = int(my_seq / 23)
    #print(f"El valor en este caso es: {valor}")
    valor *= 23
    valor = my_seq - valor;
    #print(f"El valor despues de ser modificado es {valor}")
    #print(f"lo que voy a retornar es : {letras[valor]}")

    #Ahora, ha de pasarse el numero de la secuencia
    #a una lista donde cada elemento es un numero de la secuencia
    res = [str(x) for x in str(my_seq)]
    res.append(letras[valor])
    return res

In [54]:
#Esta función crea la ciudad en la versión 0 de la simulacion.
#La ciudad es una lista de edificios y cada edificio es una lista de personas de tamaño variable.
#Input: personas de la ciudad, media de personas por edificio.
#Output: la ciudad.
def crearciudadv0(numpersonas,personasedificio):
    edificios = [] #La ciudad.
    capacidadusada=0 #Numero de personas metidas en la ciudad.
    edificioactual=0
    while (capacidadusada < numpersonas):
        column = [] #El edificio.
        esteedificio=random.randint(1,personasedificio*3) 
        '''
        Maximo de personas en el edificio. 
        Si usamos desviación tipica de capacidad de edificio, iria aqui
        '''
        contadoredificio=0 #numero de personas colocadas en este edificio
        print(f"edificio actual:{edificioactual}")
        
        #Bucle en el que se llena el edificio de personas.
        while (capacidadusada < numpersonas) and (contadoredificio < esteedificio):
            
            print(f"contador del edificio:{contadoredificio}")
            print(f"capacidad:{esteedificio}")
            
            #Insercion de persona.
            secuencia = generador_numeros() #se crea la secuencia de numeros aleatoria
            secuencia = creaDni(secuencia) #A partir de la secuencia, se crea el dni
            mi_dni = "".join(secuencia) #convierte la lista, en una sola cadena
            edad = generador_edad() #se coge un valor aleatorio para la edad de la persona
            column.append(Persona(capacidadusada,edad,edificioactual, mi_dni))
            capacidadusada+=1
            contadoredificio+=1
            
            
        #Insercion de edificio en la ciudad
        edificios.append(column)
        edificioactual+=1
        
    return edificios
def printearciudad(ciudad):        
    for j in range(len(ciudad)):
        for i in range(len(ciudad[j])):
            print(ciudad[j][i])
        print()        
        
#Funcion que dada la ciudad, y un indice, permite imprimir el edificio que corresponda a ese indice
def printearEdificio(ciudad , i):
    for i in range(len(ciudad[i])):
        print(ciudad[0][i])
    

In [55]:
ciudad=crearciudadv0(200,15)

edificio actual:0
contador del edificio:0
capacidad:7


NameError: name 'generador_numeros' is not defined

In [27]:
printearciudad(ciudad)

        

0,sano,0,73343657S
1,sano,0,51802123M
2,sano,0,22410158Q
3,sano,0,85057541E
4,sano,0,36716618Q
5,sano,0,87613118T
6,sano,0,56173857Z
7,sano,0,18120287J
8,sano,0,50221026C
9,sano,0,45662001D

10,sano,1,13606257D
11,sano,1,74173587J
12,sano,1,27247284N
13,sano,1,27255563B
14,sano,1,41160152L
15,sano,1,38565732E
16,sano,1,20186223C
17,sano,1,64137118P

18,sano,2,74622486K
19,sano,2,6310387S
20,sano,2,82103482Z
21,sano,2,34527673G
22,sano,2,80758880S
23,sano,2,5854242Y
24,sano,2,60445356E
25,sano,2,30247338S
26,sano,2,14762658Q
27,sano,2,34535652W
28,sano,2,308674Z
29,sano,2,48148477R
30,sano,2,36802243N
31,sano,2,6255256S
32,sano,2,31082617A
33,sano,2,74831176D
34,sano,2,57451042D
35,sano,2,77632847G
36,sano,2,65447471M
37,sano,2,32474455L
38,sano,2,78140850Y
39,sano,2,26471841Z
40,sano,2,42832816P
41,sano,2,56432304X
42,sano,2,43586618P

43,sano,3,66211603P
44,sano,3,62467510Q
45,sano,3,18735780A
46,sano,3,1813441Y
47,sano,3,75404366Q
48,sano,3,27285854B
49,sano,3,38017263B

50,sano,4,11

In [28]:
ciudad[0][0].estado=ciudad[0][0].estadosposibles[3]
ciudad[0][0].infectar(0.8,ciudad,7.5,3.5)


In [29]:
printearEdificio(ciudad, 0)

0,sintomatico,0,73343657S
1,sano,0,51802123M
2,sano,0,22410158Q
3,sano,0,85057541E
4,incubando,0,36716618Q
5,sano,0,87613118T
6,sano,0,56173857Z
7,sano,0,18120287J
8,sano,0,50221026C
9,sano,0,45662001D
